In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:97% !important; }</style>"))

import jupyternotify
ip = get_ipython()
ip.register_magics(jupyternotify.JupyterNotifyMagics)

import scoring as score # para hacer los reportes de puntajes
from time import time

from sklearn import preprocessing
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler  
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, RandomizedSearchCV, ShuffleSplit

properati = pd.read_csv('datos/caba_para_mapa.csv',index_col=0,error_bad_lines=False)

#tenia un nan nose porque
properati.dropna(inplace=True)

<IPython.core.display.Javascript object>

In [11]:
cant_buckets = 15

#buckets
cantidad,rango = np.histogram(properati['price_usd_per_m2'], bins=cant_buckets)
properati['categories_by_price']=pd.cut(properati['price_usd_per_m2'],rango,labels=np.arange(cant_buckets))
properati['price_range']=pd.cut(properati['price_usd_per_m2'],rango)
#lo casteo a float porque si no tira error 
properati['categories_by_price']=properati['categories_by_price'].astype(np.float64) 

#tenia un nan nose porque
properati.dropna(inplace=True)

In [12]:
#preparo set de datos
X = zip(properati['dist_a_subte'],properati['dist_a_univ'])
y = properati['categories_by_price']

scaler = StandardScaler()
scaler.fit_transform(X,y)
#reviso si quedo datos anomalos
print (np.any(np.isnan(y)) == False)
print (np.all(np.isfinite(y)) == True )

True
True


### Busco una aproximacion de hiper-parametros con random search

In [18]:
%%notify

#preparo set de datos
X = zip(properati['dist_a_subte'],properati['dist_a_univ'])
y = properati['categories_by_price']

sgd = SGDClassifier(n_jobs=-1)


alpha=np.arange(0.000001,1,0.000001)
pen =['l2','l1','elasticnet']
loss = ["hinge", "log", "modified_huber", "squared_hinge", "perceptron", "squared_loss", "huber", "epsilon_insensitive", "squared_epsilon_insensitive"]
param_grid = {"loss": loss, "penalty": pen, "alpha": alpha}
custom_cv = ShuffleSplit(n_splits=5, test_size=0.2)

# run randomized search
random_search = RandomizedSearchCV(sgd, param_distributions=param_grid,
n_iter=200,cv=custom_cv)
start = time()
random_search.fit(X, y)

print("RandomizedSearchCV duro %.2f segundos para %d candidatos a hyper-parametros."
    % (time() - start, len(random_search.cv_results_['params'])))
print("")
score.report_single(random_search.cv_results_)

RandomizedSearchCV duro 136.57 segundos para 200 candidatos a hyper-parametros.

Model with rank: 1
Mean training score: 0.280 (std: 0.004)
Mean validation score: 0.275 (std: 0.017)
Parameters: {'penalty': 'elasticnet', 'loss': 'log', 'alpha': 0.12053599999999999}

Model with rank: 2
Mean training score: 0.279 (std: 0.003)
Mean validation score: 0.275 (std: 0.016)
Parameters: {'penalty': 'elasticnet', 'loss': 'log', 'alpha': 0.89234999999999998}

Model with rank: 3
Mean training score: 0.272 (std: 0.014)
Mean validation score: 0.272 (std: 0.012)
Parameters: {'penalty': 'elasticnet', 'loss': 'log', 'alpha': 0.970082}

Model with rank: 4
Mean training score: 0.276 (std: 0.008)
Mean validation score: 0.271 (std: 0.026)
Parameters: {'penalty': 'elasticnet', 'loss': 'log', 'alpha': 0.60107600000000005}

Model with rank: 4
Mean training score: 0.277 (std: 0.007)
Mean validation score: 0.271 (std: 0.016)
Parameters: {'penalty': 'l2', 'loss': 'log', 'alpha': 0.89036999999999999}

Model with ra

<IPython.core.display.Javascript object>

In [20]:
%%notify

#preparo set de datos
X = zip(properati['dist_a_subte'],properati['dist_a_univ'])
y = properati['categories_by_price']

sgd = SGDClassifier(n_jobs=-1)
        
alpha=np.arange(0.01,1,0.01)
pen =['l2','elasticnet',"l1"]
loss = ["log", "modified_huber","huber"]
param_grid = {"loss": loss, "penalty": pen, "alpha": alpha}

custom_cv = ShuffleSplit(n_splits=5, test_size=0.2)

grid_search = GridSearchCV(sgd,param_grid=param_grid,cv=custom_cv)
start = time()
grid_search.fit(X, y)

print("GridSearchCV took %.2f seconds for %d candidate parameter settings."
    % (time() - start, len(grid_search.cv_results_['params'])))
score.report_single(grid_search.cv_results_)

GridSearchCV took 620.32 seconds for 891 candidate parameter settings.
Model with rank: 1
Mean training score: 0.282 (std: 0.008)
Mean validation score: 0.291 (std: 0.031)
Parameters: {'penalty': 'elasticnet', 'alpha': 0.65000000000000002, 'loss': 'log'}

Model with rank: 2
Mean training score: 0.282 (std: 0.011)
Mean validation score: 0.291 (std: 0.019)
Parameters: {'penalty': 'l2', 'alpha': 0.37, 'loss': 'log'}

Model with rank: 3
Mean training score: 0.282 (std: 0.009)
Mean validation score: 0.290 (std: 0.037)
Parameters: {'penalty': 'l2', 'alpha': 0.94000000000000006, 'loss': 'log'}

Model with rank: 4
Mean training score: 0.279 (std: 0.004)
Mean validation score: 0.290 (std: 0.030)
Parameters: {'penalty': 'elasticnet', 'alpha': 0.42999999999999999, 'loss': 'log'}

Model with rank: 5
Mean training score: 0.282 (std: 0.014)
Mean validation score: 0.289 (std: 0.019)
Parameters: {'penalty': 'elasticnet', 'alpha': 0.90000000000000002, 'loss': 'log'}

Model with rank: 6
Mean training sc

<IPython.core.display.Javascript object>